# 🛰️ StreetView Safety Analyzer

In [ ]:
# Install dependencies (uncomment if needed)
# !pip install torch transformers pillow pandas google-streetview geopandas shapely folium ipyleaflet

import os
import glob
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape, Point
from datetime import datetime
from transformers import AutoProcessor, AutoModelForVision2Seq
from PIL import Image
import torch
from ipyleaflet import Map, DrawControl
from IPython.display import display


In [ ]:
draw_output = {}
m = Map(center=(-6.2, 106.8), zoom=13)

def handle_draw(_, action, geo_json):
    global draw_output
    draw_output = geo_json
    print("ROI selected and saved.")

draw_control = DrawControl(polygon={"shapeOptions": {"color": "#f06eaa"}})
draw_control.on_draw(handle_draw)
m.add_control(draw_control)
display(m)


In [ ]:
from random import uniform

def generate_random_points(geojson, n=10):
    polygon = shape(geojson['geometry'])
    points = []
    while len(points) < n:
        lon = uniform(*polygon.bounds[::2])
        lat = uniform(*polygon.bounds[1::2])
        pt = Point(lon, lat)
        if polygon.contains(pt):
            points.append(pt)
    return points

gdf = gpd.GeoDataFrame(geometry=generate_random_points(draw_output, 10))
gdf["lat"] = gdf.geometry.y
gdf["lon"] = gdf.geometry.x
gdf


In [ ]:
image_folder = f"images/session_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
os.makedirs(image_folder, exist_ok=True)
gdf.to_csv(f"{image_folder}/points.csv", index=False)


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")
model = AutoModelForVision2Seq.from_pretrained(
    "llava-hf/llava-1.5-7b-hf", device_map="auto", torch_dtype=torch.float16
)

def describe_with_llava(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, text="Describe the street safety features in this image.", return_tensors="pt").to(device)
    generated = model.generate(**inputs, max_new_tokens=100)
    return processor.batch_decode(generated, skip_special_tokens=True)[0]


In [ ]:
latest_dir = sorted(glob.glob("images/session_*"))[-1]
image_paths = sorted(glob.glob(f"{latest_dir}/*.jpg"))

results = []
for path in image_paths:
    caption = describe_with_llava(path)
    results.append({"image": os.path.basename(path), "caption": caption})

df = pd.DataFrame(results)
df.to_csv(f"{latest_dir}/captions.csv", index=False)
df.head()


In [ ]:
def extract_safety_flags(text):
    return {
        "crosswalk": "crosswalk" in text.lower(),
        "light": "light" in text.lower() or "signal" in text.lower(),
        "pothole": "pothole" in text.lower() or "crack" in text.lower(),
        "daylight": "daylight" in text.lower() or "sun" in text.lower()
    }

flags_df = df["caption"].apply(extract_safety_flags).apply(pd.Series)
final_df = pd.concat([df, flags_df], axis=1)
final_df.to_csv(f"{latest_dir}/safety_flags.csv", index=False)
final_df
